In [1]:
import os
import sys
import numpy as np
import pickle
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import pandas as pd



In [2]:
import pandas as pd
import jieba

In [3]:
data = pd.read_csv('../data/atec_nlp_sim_train.csv',sep='\t',encoding='utf8', names=['id', 'q1', 'q2', 'label'] )

In [4]:
data.head()

,id,q1,q2,label
0,1,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1
1,2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
2,3,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0
3,4,如何得知关闭借呗,想永久关闭借呗,0
4,5,花呗扫码付钱,二维码扫描可以用花呗吗,0


In [5]:
data.label.value_counts()

0    83792
1    18685
Name: label, dtype: int64

In [7]:
83792/18685

4

In [13]:
data['q1_cut']=data['q1'].map(lambda x:list(jieba.cut(x)))
data['q2_cut']=data['q2'].map(lambda x:list(jieba.cut(x)))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 7.165 seconds.
Prefix dict has been built succesfully.


In [15]:
data.to_csv('../data/cut.csv')

In [7]:
content =list(data.q1_cut)+list(data.q2_cut)


model = Word2Vec(content, size=config['w2v_vec_dim'], window=5, min_count=5, workers=multiprocessing.cpu_count()
                 )
model.save(config['w2v_content_word_model'])

In [8]:
weights = model.wv.syn0
vocab = dict([(k, v.index + 1) for k, v in model.wv.vocab.items()])
vocab['<-UNKNOW->'] = len(vocab) + 1
embed_weights = np.zeros(shape=(weights.shape[0] + 2, weights.shape[1]))
embed_weights[1:weights.shape[0] + 1] = weights
unk_vec = np.random.random(size=weights.shape[1]) * 0.5
pading_vec = np.random.random(size=weights.shape[1]) * 0
embed_weights[weights.shape[0] + 1] = unk_vec - unk_vec.mean()
embed_weights[0] = pading_vec

#pickle.dump(vocab, open(config['word_embed_dict'], "wb"))

config['vocab_size'] = embed_weights.shape[0]
config['w2v_vec_dim'] = embed_weights.shape[1]
yaml.dump(config, open('config.yaml', 'w'))

np.save(config['word_embed_weight'], embed_weights)
print(embed_weights.shape)
print('save')

(2552, 256)
save


/home/bigdata/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  if __name__ == '__main__':


In [9]:
word_voc = vocab

In [10]:



def padding_id(ids, padding_token=0, padding_length=None):
    if len(ids) > padding_length:
        return ids[:padding_length]
    else:
        return ids + [padding_token] * (padding_length - len(ids))


def word2id(contents):
    ''' contents  list
    '''
#     contents = str(contents)
#     contents = contents.split()

    ids = [word_voc[c] if c in word_voc else len(word_voc) for c in contents]

    return padding_id(ids, padding_token=0, padding_length=config['word_maxlen'])


def data2id():

    data = read_df(config['train_cut'])
    data['title'] = data.title.map(lambda x: word2id(x))
    data['content'] = data.content.map(lambda x: word2id(x))
    print(data.head(4))
    data.to_pickle(config['train_cut_id_dump'])


In [12]:
data['q1_cut_id'] = data['q1_cut'].map(lambda x: word2id(x))
data['q2_cut_id'] = data['q2_cut'].map(lambda x: word2id(x))

In [13]:
data.head()

,id,q1,q2,label,q1_cut,q2_cut,q1_cut_id,q2_cut_id
0,﻿1,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1,"[﻿, 怎么, 更改, 花, 呗, 手机号码]","[我, 的, 花, 呗, 是, 以前, 的, 手机号码, ，, 怎么, 更, 改成, 现在,...","[2551, 10, 233, 2, 1, 216, 0, 0, 0, 0, 0, 0, 0...","[5, 4, 2, 1, 19, 159, 4, 216, 8, 10, 986, 603,..."
1,2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,"[也, 开, 不了, 花, 呗, ，, 就, 这样, 了, ？, 完事, 了]","[真的, 嘛, ？, 就是, 花, 呗, 付款]","[103, 160, 32, 2, 1, 8, 74, 623, 7, 139, 2479,...","[802, 131, 139, 225, 2, 1, 35, 0, 0, 0, 0, 0, ..."
2,3,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,"[花, 呗, 冻结, 以后, 还, 能, 开通, 吗]","[我, 的, 条件, 可以, 开通, 花, 呗, 借款, 吗]","[2, 1, 110, 178, 11, 23, 20, 9, 0, 0, 0, 0, 0,...","[5, 4, 186, 14, 20, 2, 1, 120, 9, 0, 0, 0, 0, ..."
3,4,如何得知关闭借呗,想永久关闭借呗,0,"[如何, 得知, 关闭, 借, 呗]","[想, 永久, 关闭, 借, 呗]","[55, 2551, 53, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[70, 574, 53, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,5,花呗扫码付钱,二维码扫描可以用花呗吗,0,"[花, 呗, 扫码, 付钱]","[二维码, 扫描, 可以, 用花, 呗, 吗]","[2, 1, 311, 556, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[210, 868, 14, 33, 1, 9, 0, 0, 0, 0, 0, 0, 0, ..."


In [44]:
import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing import sequence
from keras.regularizers import l2
from keras import backend as K
from keras.engine.topology import Layer
from keras.backend.tensorflow_backend import set_session
import time
from tensorflow.contrib import learn
import yaml
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from keras import backend as K

params = yaml.load(open('params.yaml', 'r'))
config = yaml.load(open('config.yaml', 'r'))

In [25]:
def convs_block(data, convs=[3,4,5], f=256, name="conv_feat"):
    pools = []
    for c in convs:
        conv = Activation(activation="relu")(BatchNormalization()(
            Conv1D(filters=f, kernel_size=c, padding="valid")(data)))
        pool = GlobalMaxPool1D()(conv)
        pools.append(pool)
    return concatenate(pools, name=name)


def convs_block2(data, convs=[3, 4, 5], f=256, name="conv_feat"):
    pools = []
    for c in convs:
        conv = Activation(activation="relu")(BatchNormalization()(
            Conv1D(filters=f, kernel_size=c, padding="valid")(data)))
        conv = MaxPool1D(pool_size=10)(conv)
        conv = Activation(activation="relu")(BatchNormalization()(
            Conv1D(filters=f, kernel_size=c, padding="valid")(conv)))

        pool = GlobalMaxPool1D()(conv)
        pools.append(pool)
    return concatenate(pools, name=name)


def get_textcnn(seq_length, embed_weight, pretrain=False):
    content = Input(shape=(seq_length,), dtype="int32")
    if pretrain:
        embedding = Embedding(name='word_embedding', input_dim=config[
                              'vocab_size'], weights=[embed_weight], output_dim=config['w2v_vec_dim'], trainable=False)
    else:
        embedding = Embedding(name='word_embedding', input_dim=config[
                              'vocab_size'], output_dim=config['w2v_vec_dim'], trainable=True)
    trans_content = Activation(activation="relu")(
        BatchNormalization()((TimeDistributed(Dense(256))(embedding(content)))))
    feat = convs_block(trans_content)
    dropfeat = Dropout(0.2)(feat)
    fc = Activation(activation="relu")(
        BatchNormalization()(Dense(256)(dropfeat)))
    output = Dense(2, activation="softmax")(fc)
    model = Model(inputs=content, outputs=output)
    model.compile(loss='categorical_crossentropy',
                  optimizer="adam", metrics=['accuracy'])
    model.summary()
    return model

In [26]:
import sys

import numpy as np
import pandas as pd
import pickle
import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing import sequence
from keras.regularizers import l2
from keras import backend as K
from keras.engine.topology import Layer
from keras.backend.tensorflow_backend import set_session
#from recurrentshop import *
import time
from tensorflow.contrib import learn
import yaml
from sklearn.model_selection import train_test_split
import keras.backend as K
from keras.callbacks import TensorBoard
params = yaml.load(open('params.yaml', 'r'))
config = yaml.load(open('config.yaml', 'r'))

from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score



def score(label, pred, gate=0.5):

    if len(label.shape) == 1:
        p = (pred > gate).astype("int")
        p = np.squeeze(p)
        l = label
    else:
        p = np.argmax(pred, axis=1)
        l = np.argmax(label, axis=1)

    #print(confusion_matrix(l, p).view())
    pre_score = precision_score(l, p, pos_label=1, average='binary')
    rec_score = recall_score(l, p, pos_label=1, average='binary')
    f_score = f1_score(l, p)
    return pre_score, rec_score, f_score

In [27]:
q1_id = np.array(list(data.q1_cut_id.values))
q2_id = np.array(list(data.q2_cut_id.values))

In [28]:
q1_id.shape

(39346, 20)

In [29]:
np.concatenate([q1_id,q2_id],1).shape

(39346, 40)

In [30]:
data['label'] = data['label'].apply(
        lambda x: int(x ==1))  # 人写的是1 机器写的是0
X = np.concatenate([q1_id,q2_id],1)
Y = keras.utils.to_categorical(list(data.label), num_classes=2)

In [31]:
x_train, x_dev, y_train, y_dev = train_test_split(X, Y,    test_size=0.1,     random_state=2008)


In [32]:
def train_batch_generator(x_source, y_source, batch):
    while True:
        batch_list_x = []
        batch_list_y = []
        for line, y in zip(x_source, y_source):
            x = line.astype('float32')
            batch_list_x.append(x)
            batch_list_y.append(y)
            if len(batch_list_x) == batch:
                yield (np.array(batch_list_x), np.array(batch_list_y))
                batch_list_x = []
                batch_list_y = []

In [35]:
model = get_textcnn(config['word_maxlen']*2,
                        embed_weights, pretrain=False)

for i in range(15):
    if i == 8:
        K.set_value(model.optimizer.lr, 0.0001)

    model.fit_generator(
        train_batch_generator(x_train, y_train, config['batch_size']),
        epochs=1,
        steps_per_epoch=int(x_train.shape[0] / config['batch_size']),
        validation_data=(x_dev, y_dev),
    
    )

    pred = model.predict(x_dev, batch_size=config['batch_size'])
    pre, rec, f1 = score(y_dev, pred)

    print('p r f1 ', pre, rec, f1)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
word_embedding (Embedding)      (None, 40, 256)      653312      input_4[0][0]                    
__________________________________________________________________________________________________
time_distributed_4 (TimeDistrib (None, 40, 256)      65792       word_embedding[0][0]             
__________________________________________________________________________________________________
batch_normalization_20 (BatchNo (None, 40, 256)      1024        time_distributed_4[0][0]         
__________________________________________________________________________________________________
activation

KeyboardInterrupt: 

In [60]:
def get_textrnn(seq_length, embed_weight, pretrain=False):
    main_input = Input(shape=(seq_length,), dtype='float64')
    if pretrain:
        embedding = Embedding(name='word_embedding', input_dim=config[
            'vocab_size'], weights=[embed_weight], output_dim=config['w2v_vec_dim'], trainable=False)
    else:
        embedding = Embedding(name='word_embedding', input_dim=config[
            'vocab_size'], output_dim=config['w2v_vec_dim'], trainable=True)
    out = embedding(main_input)
    out = Activation(activation="relu")(
        BatchNormalization()((TimeDistributed(Dense(256))(out))))
    print('Build model...')
    out = Bidirectional(GRU(256))(out)
    #out= BatchNormalization(out)
    out = Dropout(0.3)(out)
    fc = Activation(activation="relu")(
        BatchNormalization()(Dense(256)(out)))
    main_output = Dense(config['number_classes'],
                        activation='softmax')(fc)

    model = Model(inputs=main_input, outputs=main_output)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:

model = get_textrnn(config['word_maxlen']*2, embed_weights, pretrain=False)

for i in range(15):
    if i == 8:
        K.set_value(model.optimizer.lr, 0.0001)

    model.fit_generator(
        train_batch_generator(x_train, y_train, config['batch_size']),
        epochs=1,
        steps_per_epoch=int(x_train.shape[0] / config['batch_size']),
        validation_data=(x_dev, y_dev),
    
    )

    pred = model.predict(x_dev, batch_size=config['batch_size'])
    pre, rec, f1 = score(y_dev, pred)

    print('p r f1 ', pre, rec, f1)


Build model...
Epoch 1/1
276/276 [==============================] - 682s 2s/step - loss: 0.5495 - acc: 0.7704 - val_loss: 0.5360 - val_acc: 0.7629
('p r f1 ', 0.34909090909090912, 0.11294117647058824, 0.17066666666666669)
Epoch 1/1
276/276 [==============================] - 671s 2s/step - loss: 0.4976 - acc: 0.7841 - val_loss: 0.5197 - val_acc: 0.7776
('p r f1 ', 0.44680851063829785, 0.12352941176470589, 0.19354838709677422)
Epoch 1/1
276/276 [==============================] - 673s 2s/step - loss: 0.4629 - acc: 0.7953 - val_loss: 0.5173 - val_acc: 0.7751
('p r f1 ', 0.44444444444444442, 0.16470588235294117, 0.24034334763948498)
Epoch 1/1
276/276 [==============================] - 653s 2s/step - loss: 0.4329 - acc: 0.8070 - val_loss: 0.5357 - val_acc: 0.7677
('p r f1 ', 0.41160220994475138, 0.17529411764705882, 0.24587458745874588)
Epoch 1/1
276/276 [==============================] - 663s 2s/step - loss: 0.4004 - acc: 0.8234 - val_loss: 0.6243 - val_acc: 0.6574
('p r f1 ', 0.28534482758